In [1]:
# ! pip install pandas web3 hexbytes rlp fastlz clickhouse-connect
# ! pip install python-dotenv

### Readme
FastLZ needs Python version 3.9x or lower, make sure your environment is using a later python version

In [2]:
import pandas as pd
from web3 import Web3
from hexbytes import HexBytes
import ast
import rlp
from rlp.sedes import Binary, big_endian_int, binary, List
from eth_utils import to_bytes, to_hex
import fastlz
import sys
import os
import dotenv
dotenv.load_dotenv()
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

In [3]:
#Configuration
chain_mappings_list = [
        {'schema_name': 'op', 'display_name': 'OP Mainnet', 'chain_id': 10},
        # {'schema_name': 'base', 'display_name': 'Base'},
        # {'schema_name': 'mode', 'display_name': 'Mode'},
        # {'schema_name': 'fraxtal', 'display_name': 'Fraxtal'},
    # Add more mappings as needed
]

days_of_data = 7

#FastLZ Regression Metrics
# Specs - https://specs.optimism.io/fjord/exec-engine.html?search=#fjord-l1-cost-fee-changes-fastlz-estimator
intercept = -42_585_600
fastlzCoef = 836_500
minTransactionSize = 100
scaled_by = 1e6


In [4]:
# Test transaction receipt
from web3 import Web3
op_rpc = os.getenv("OP_PUBLIC_RPC")
w3 = Web3(Web3.HTTPProvider(op_rpc))

tx_test = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
tx = w3.eth.get_transaction(tx_test)
txr = w3.eth.get_transaction_receipt(tx_test)
# # txraw = w3.eth.get_raw_transaction(tx_test)
print(tx)
# print(txr)
# # print(txraw)

AttributeDict({'blockHash': HexBytes('0x694fb0156a5719a8ee75ebf2568e8a8d55899db9312127f0d4fe5001c06ac397'), 'blockNumber': 120731426, 'chainId': 10, 'from': '0x899e837095a0F3CC62FB05998559Df90F26A1F46', 'gas': 69534, 'gasPrice': 4307539, 'hash': HexBytes('0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'), 'input': HexBytes('0xa9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a7740000'), 'nonce': 206, 'r': HexBytes('0x6727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357'), 's': HexBytes('0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'), 'to': '0xdC6fF44d5d932Cbd77B52E5612Ba0529DC6226F1', 'transactionIndex': 4, 'type': 0, 'v': 55, 'value': 0})


In [5]:
# may not sufficent due to missing transaction signature fields

# Get L2 Txs from Clickhouse / Goldsky
query = '''
        SELECT @chain_id@ as chain_id, nonce, gas, max_fee_per_gas, max_priority_fee_per_gas,
                to_address as to, value, input, block_timestamp, block_number, hash, receipt_gas_used
        FROM @chain_db_name@_transactions
        WHERE gas_price > 0
        AND block_timestamp < DATE_TRUNC('day',NOW())
        AND block_timestamp >= DATE_TRUNC('day',NOW() - interval '@num_days@ days')

        SETTINGS max_execution_time = 3000
'''

# txs_df

In [6]:
# Process transactions and RLP encode
#https://ethereum.org/en/developers/docs/transactions/

# NOTE THE RLP ENCODING IS NOT 1:1 WITH ETHERSCAN YET (but it's ~close-ish)
def process_and_encode_transaction(row):
    to_hexstr = 'eee'
    try:
        # Check if "to" field is None or empty
        to_field = row["to"]
        if isinstance(to_field, bytes) and to_field == b'\x00' * len(to_field):
            to_hexstr = b''
        else:
            to_hexstr = to_bytes(hexstr=to_field.decode('utf-8')) if to_field is not None else b''
        tx_params = {
            'chainId': row['chain_id'],
            'nonce': row['nonce'],
            'maxPriorityFeePerGas': row['max_priority_fee_per_gas'],
            'maxFeePerGas': row['max_fee_per_gas'],
            'gas': row['gas'],
            'to': to_hexstr,
            'value': to_bytes(row['value']),
            'input': HexBytes(row['input']),
            'accessList': row['access_list'],  # This is assumed to be an empty list or properly formatted
            'v': row['v'],
            'r': HexBytes(row['r']),
            's': HexBytes(row['s'])
        }

        transaction = [
            tx_params['chainId'],
            tx_params['nonce'],
            tx_params['gas'],
            tx_params['maxFeePerGas'],
            tx_params['maxPriorityFeePerGas'],
            tx_params['to'],
            tx_params['value'],
            tx_params['input'],
            tx_params['accessList'],
            tx_params['v'],
            tx_params['r'],
            tx_params['s']
        ]

        encoded_tx = rlp.encode(transaction)
        return Web3.to_hex(encoded_tx), len(encoded_tx)
    except ValueError as e:
        print("Error:", e)
        print("Failed Transaction Info:")
        print(row)
        print(to_hexstr)
        return None, None

# Function to compress transaction data
def compress_transaction(encoded_transaction):

    hex_string = encoded_transaction[2:]
    # Convert the hexadecimal string to bytes
    byte_string = bytes.fromhex(hex_string)
    compressed_data = fastlz.compress(byte_string)

    return compressed_data.hex(), len(compressed_data)
# Define a function to apply to each row of the DataFrame
def process_and_compress_transaction(row):
    encoded_tx = row['encoded_transaction']
    compressed_tx, len_tx = compress_transaction(encoded_tx)
    return compressed_tx, len_tx
# print(txs_df['encoded_transaction'][0])
# print(len(txs_df['encoded_transaction'][0]))

In [ ]:
dfs = []
for chain in chain_mappings_list:
        query_map = query

        query_map = query_map.replace("@chain_db_name@", chain['schema_name'])
        query_map = query_map.replace("@chain_id@", str(chain['chain_id']))
        query_map = query_map.replace("@num_days@", str(days_of_data))

        result_df = client.query_df(query_map)

        # Add Dummy Signature and fields
        result_df['access_list'] = '[]'
        result_df['access_list'] = result_df['access_list'].apply(ast.literal_eval)
        result_df['r'] = '0xf267a8b78655826edfcb5cd4b852804c0ae1e1ddb14021b79a117a184f87678a'
        result_df['s'] = '0x5693e2fcdf1280fc8e957e900dece6edd9bcd201a233d3f1edb58d64d54e7fa9'
        result_df['v'] = '56'

        # Assuming `txs_df` is your DataFrame
        result_df[['encoded_transaction', 'len_encoded_transaction']] = result_df.apply(process_and_encode_transaction, axis=1, result_type='expand')
        # Apply compression to each transaction in the DataFrame
        result_df[['compressed_transaction', 'compressed_transaction_length']] = result_df.apply(process_and_compress_transaction, axis=1, result_type='expand')
        # Calculate estimated size for each row
        result_df['estimatedSize_raw'] = result_df.apply(lambda row: (intercept + (row['compressed_transaction_length'] * fastlzCoef)) / scaled_by, axis=1)
        # Calculate minimum value for 'estimatedSize' column
        result_df['estimatedSize'] = result_df.apply(lambda row: max(minTransactionSize, row['estimatedSize_raw']), axis=1)

        # Agg L2
        # Convert block_timestamp to date (truncate to day)
        result_df['block_date'] = pd.to_datetime(result_df['block_timestamp']).dt.date
        grouped_df = result_df.groupby(['block_date', 'chain_id'])
        # Define aggregation functions
        agg_functions = {
                'len_encoded_transaction': ['sum', 'mean', 'count'],
                'estimatedSize': ['sum', 'mean']
        }
        # Perform aggregation
        aggregated_df = grouped_df.agg(agg_functions).reset_index()
        # Rename columns for clarity
        aggregated_df.columns = ['block_date', 'chain_id', 
                                'total_len_encoded_transaction', 'average_len_encoded_transaction', 'transaction_count',
                                'total_estimatedSize', 'average_estimatedSize']
                
        dfs.append(aggregated_df)

aggregated_df = pd.concat(dfs)

In [17]:
agg_cols = ['average_len_encoded_transaction','average_estimatedSize','transaction_count']
total_aggregated_df = aggregated_df[['chain_id'] + agg_cols].groupby(['chain_id']).mean([])
total_aggregated_df

,average_len_encoded_transaction,average_estimatedSize,transaction_count
chain_id,,,
10,1361.54135,464.479114,496559.285714


In [11]:
from datetime import datetime
# Generate current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# Define the file path
file_path = f"outputs/l2_output_{current_timestamp}.csv"
# Save the DataFrame to CSV
aggregated_df.to_csv(file_path, index=False)
print(f"DataFrame saved to: {file_path}")

DataFrame saved to: outputs/l2_output_20240531_210144.csv


In [9]:
# Pull aggregate L1 data

In [10]:
# Generate L2 : L1 ratio metrics